<a href="https://colab.research.google.com/github/CodyDrolet/Airline/blob/main/Airline_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -q 'git+https://github.com/drscook/helpers'
! pip install -q --upgrade numpy pandas scikit-learn matplotlib shap jupyter-autotime
# restart kernel so upgrade takes effect
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 87.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

{'status': 'ok', 'restart': True}

In [1]:
import plotly.express as px
import pandas as pd
import io
import missingno as msno
from google.colab import auth
from helpers.common_imports import *
from helpers import utilities as ut
bq = ut.BigQuery()
auth.authenticate_user()
from google.cloud import bigquery
client = bigquery.Client('tarletondatascience2022.drolet')

### Airlines


Airline Project

Cody Drolet

Data Warehousing

Showcasing


*   Different Types of joins
*   Python visualization techniques
*   Data Wrangling
*   Feature Engineering










# Introduction



The goal is to use Airline flight data to try to see what affects airlines' fuel efficiency.



**What question(s) do you want to answer?**



Airline companies spend a lot of money on fuel and the more fuel-efficient airlines tend to save more money per flight. With flight data and weather data readily available the question raises does the weather have an effect on airlines' fuel efficiency, which airline is more fuel efficient and why? Does precipitation have an effect on fuel efficiency for airplanes?



**Why does this topic interest you?  Why should someone else be interested?**



This topic interests me because I have family and friends in the airline industry and wanted to get something going to get my foot in the door.





## Data

**What data sets are you using?**



I am using weather,weather_geo,fuel_usage, airline, airport, and flight data.



**Who produced it?**



https://www.kaggle.com/datasets/usdot/flight-delays



https://www.transtats.bts.gov/fuel.asp



I got fuel usage with this data set and manually put it into an excel sheet, the weather and weather geo were provided to me by big_querys public data set. ghcnd_2015



https://console.cloud.google.com/bigquery?project=tarletondatascience2022&ws=!1m5!1m4!4m3!1sbigquery-public-data!2sghcn_d!3sghcnd_2015



**When & where was it collected?**



This data was collected in the US in 2015 at every airport and weather station.



**Limitations of the data?**



Only had fuel_usage from 2015.



Weather- Some missing values for temp and prcp and I wasn't able to utilize wind.



Airline & fuel usage had 13 airlines whereas the flight data had 14 airlines.



Flight data didn't give me information about the type of plane that would have some effect on fuel efficiency.





In [3]:
Airlines = 'tarletondatascience2022.drolet.Airline2'#this table just gives me for each airline its
#corresponding code
Airports = 'tarletondatascience2022.drolet.airports'
#this table gives me every Airport in the US and its Latitude and Longitude
fuel_usage = 'tarletondatascience2022.drolet.Fuel_usage2'
#this table gives me an airlines fuel usage broken down by month
Flights = 'tarletondatascience2022.drolet.Flights2'
#this table is flights for the year 2015 broken down by day
#weather = 'tarletondatascience2022.drolet.weather3'
#weather2 = 'bigquery-public-data.ghcn_d.ghcnd_2015'
weather_tbl  = 'bigquery-public-data.ghcn_d.ghcnd_2015'
#this table gives me the weather at a radio station every day
weather_geo = 'tarletondatascience2022.drolet.WeatherGeo'
#this table gives me the lat and long of every weather station in the US
#the resulting table after join weather,weather geo, airports, flights
#flights_distance = 'tarletondatascience2022.drolet.flights_distance2'
#this gives me flight_distance for every airline for every month as well as its corresponding airport weather

## Methodology


**Joins & Unions**



I did a cross-join between weather_geo and airport data to get the closest airport to the closest weather station. Then I joined it to the flight and weather data. This table was at the day level so I aggregated everything up to the month level once that was done then I joined it to my already joined monthly data of fuel usage and airline to get fuel usage by month for every airline and flight. This is in the long format where it shows me information by airline differing based on month and airport.



I checked the count on every table after every join and had reasonable reasons why tables were shrinking or growing. The join when I joined the airline to flight shrunk because there were 13 airlines in the airline table and 14 in the flight table.



**Feature Engineering**



I dropped a good amount of tables for simplicity reasons and effect reasons the main point was to find prcp and their effects on fuel efficiency. I also dropped one airline because in the fuel usage table their numbers were very skewed and it brought into question the accuracy of the reports of their fuel usage.



I rescaled everything to the monthly level, and calculated flight distance, fuel efficiency, and avg rainfall .



**Feature Description**



avg_temp - is the tmax+tmin /2 This is the average temp at every airport for every month



flight_distance- this will give me the distance flown for every airline by month.                                                              sum(DISTANCE) over (partition by month,airline) as flight_distance



fuel_efficency- max(flight_distance/fuel_usage) as fuel_efficency



In the end, I had repeating values because I partitioned by everything which was making the visualization bad so I did max of everything and then grouped by the airline, airport, and month to get exactly one row for every distinct combo of those. To get the distance between two latitude and longitude points I did st_distance command.



sum(DISTANCE) over (partition by month,airline) as flight_distance,     



avg(prcp) over (partition by month,iata_code,airline) as prcp



 ST_distance(Point, air_point) as weather_distance





##Code


In [7]:
Airlines = 'tarletondatascience2022.drolet.Airline2'
cook_demo_qry = f"""
select
    *
from
    {Airlines}
limit 10
"""
Airlines = 'tarletondatascience2022.drolet.Airline2'#this table just gives me for each airline its
#corresponding code
Airports = 'tarletondatascience2022.drolet.airports'
#this table gives me every Airport in the US and its Latitude and Longitude
fuel_usage = 'tarletondatascience2022.drolet.Fuel_usage2'
#this table gives me an airlines fuel usage broken down by month
Flights = 'tarletondatascience2022.drolet.Flights2'
#this table is flights for the year 2015 broken down by day
weather = 'tarletondatascience2022.drolet.weather2'
weather2 = 'bigquery-public-data.ghcn_d.ghcnd_2015'
#this table gives me the weather at a radio station every day
weather_geo = 'tarletondatascience2022.drolet.WeatherGeo'
#this table gives me the lat and long of every weather station in the US
#the resulting table after join weather,weather geo, airports, flights
flights_distance = 'tarletondatascience2022.drolet.flights_distance2'
#this gives me flight_distance for every airline for every month as well

airlines_tbl = 'tarletondatascience2022.drolet.Airline2'
airports_tbl = 'tarletondatascience2022.drolet.airports'
flights_tbl  = 'tarletondatascience2022.drolet.Flights2'
stations_tbl = 'bigquery-public-data.ghcn_d.ghcnd_stations'
weather_tbl  = 'bigquery-public-data.ghcn_d.ghcnd_2015'

# Because prcp & snow are much more complete than other weather elements
# and they probably have the biggest affect on flights, we'll focus on them.
# Because they have similiar affects, we'll combine them into one value,
# correcting for the fact that snow is given in mm but prcp is given in tenth of mm.
# We say a station "qualifies" if it reported prcp in at least 90% of days.
# The final qualify statement excludes non-qualifying stations
weather = f"""
select
    id,
    date,
    sum(case when lower(element) = 'snow' then value else value / 10 end) as prcp,
from
    {weather_tbl}
where
    qflag is null
    and left(id, 2) = 'US'
    and lower(element) in ('prcp', 'snow')
group by
    id, date
qualify
    count(*) over (partition by id) / 365 > 0.90
"""


# Join airports to the closest qualifying weather station,
# removing airports more than 10km from any qualifying station
airports_stations = f"""
select
    A.*,
    id,
    st_distance(air_point, st_geogpoint(longitude, latitude)) as dist,
from
    {airports_tbl} as A
cross join
    {stations_tbl} as S
where
    air_point is not null
    and id in (select distinct id from ({weather}))
qualify
    dist = min(dist) over (partition by iata_code)
    and dist < 10000
"""


# Join flights to weather data via their origin airport's weather station
# Note use of format_date("%B", date(year, month, day)) to get month name more easily
flights_weather = f"""
select
    F.*,
    O.airport as origin_name,
    O.city as origin_city,
    O.state as origin_state,
    O.id as origin_station_id,
    O.dist as origin_station_dist,
    W.prcp as origin_prcp,
from (
    select
        date(year, month, day) as date,
        month,
        format_date("%B", date(year, month, day)) as month_name,
        airline,
        flight_number,
        distance,
        origin_airport as origin_iata,
    from
        {flights_tbl}
    ) as F
join (
    {airports_stations}
    ) as O
on
    F.origin_iata = O.iata_code
join (
    {weather}
    ) as W
on
    W.id = O.id and W.date = F.date
"""

flights_distance = f"""
select
    *,
    monthly_distance/fuel_usage as fuel_efficency,
    A.AIRLINE as airline_name
from
(
    select
        month,origin_iata,origin_city,origin_state,origin_station_id,airline,month_name,
        max(monthly_distance) as monthly_distance,
        max(monthly_prcp) as monthly_prcp,
        --max(flight_number)  as flight_number
    from
    (
        select
            *,
            sum(distance) over (partition by month, airline) as monthly_distance,
            cast(sum(origin_prcp) over (partition by airline, month,origin_iata) as int) as monthly_prcp
        from
            (
                {flights_weather}
            ) as F
    )
    group by month,origin_iata,origin_city,origin_state,origin_station_id,airline,month_name
)
join
    {fuel_usage} as U
on
    airline = U.IATA_CODE
    and
    month_name = U.month
join
    {Airlines} as A
on
    A.IATA_CODE = U.IATA_CODE
where
A.AIRLINE != 'Atlantic Southeast Airlines'
order by monthly_prcp desc
"""
f = bq.qry_to_df(flights_distance)

## Visualizations

In [5]:
airlines_tbl = 'tarletondatascience2022.drolet.Airline2'
airports_tbl = 'tarletondatascience2022.drolet.airports'
fuel_tbl     = 'tarletondatascience2022.drolet.Fuel_usage2'
flights_tbl  = 'tarletondatascience2022.drolet.Flights2'
stations_tbl = 'bigquery-public-data.ghcn_d.ghcnd_stations'
weather_tbl  = 'bigquery-public-data.ghcn_d.ghcnd_2015'

# Because prcp & snow are much more complete than other weather elements
# and they probably have the biggest affect on flights, we'll focus on them.
# Because they have similiar affects, we'll combine them into one value,
# correcting for the fact that snow is given in mm but prcp is given in tenth of mm.
# We say a station "qualifies" if it reported prcp in at least 90% of days.
# The final qualify statement excludes non-qualifying stations
weather = f"""
select
    id,
    date,
    sum(case when lower(element) = 'snow' then value else value / 10 end) as prcp,
from
    {weather_tbl}
where
    qflag is null
    and left(id, 2) = 'US'
    and lower(element) in ('prcp', 'snow')
group by
    id, date
qualify
    count(*) over (partition by id) / 365 > 0.90
"""


# Join airports to the closest qualifying weather station,
# removing airports more than 10km from any qualifying station
airports_stations = f"""
select
    A.*,
    id,
    st_distance(air_point, st_geogpoint(longitude, latitude)) as dist,
from
    {airports_tbl} as A
cross join
    {stations_tbl} as S
where
    air_point is not null
    and id in (select distinct id from ({weather}))
qualify
    dist = min(dist) over (partition by iata_code)
    and dist < 10000
"""


# Join flights to weather data via their origin airport's weather station
# Note use of format_date("%B", date(year, month, day)) to get month name more easily
flights_weather = f"""
select
    F.*,
    O.iata_code as origin_iata_code,
    O.airport as origin_name,
    O.city as origin_city,
    O.state as origin_state,
    O.id as origin_station_id,
    O.dist as origin_station_dist,
    W.prcp as origin_prcp,
from (
    select
        date(year, month, day) as date,
        month,
        format_date("%B", date(year, month, day)) as month_name,
        airline as airline_iata,
        flight_number,
        distance,
        origin_airport as origin_iata,
    from
        {flights_tbl}
    ) as F
join (
    {airports_stations}
    ) as O
on
    F.origin_iata = O.iata_code
join (
    {weather}
    ) as W
on
    W.id = O.id and W.date = F.date
"""

qry = f"""
select
    C.airline as airline_name,
    A.*,
    B.fuel_usage,
    distance / B.fuel_usage as fuel_efficiency,
    A.prcp_sum / flights as prcp_avg,
    A.prcp_ct / flights * 100 as prcp_pct,
from (
    select
        airline_iata,
        month,
        max(month_name) as month_name,
        count(*) as flights,
        1.0 * sum(distance) as distance,  -- multiplying by 1.0 casts int to float so viz works better later
        --cast(sum(distance) as float) as distance,  -- multiplying by 1.0 casts int to float so viz works better later
        sum(origin_prcp) as prcp_sum,
        sum(case when origin_prcp > 0 then 1 else 0 end) as prcp_ct,
    from (
        {flights_weather}
        )
    group by
        airline_iata,
        month
    ) as A
join
    {fuel_tbl} as B
on
    A.airline_iata = B.iata_code and A.month_name = B.month
join
    {airlines_tbl} as C
on
    B.iata_code = C.iata_code
"""

df = bq.qry_to_df(qry)
df

import plotly.express as px
# removes Atlantic Southeast & Skywest which have suspiciously high fuel_efficiency (10-1000 times better than everyone else)
# and sorts so legend and animations are in the correct order
data = df.query('fuel_efficiency<1000').sort_values(['airline_name', 'month'])

# Whether we look at percent of flights with prcp at origin OR total mm of prcp at origin,
# we don't see a correlation between prcp and fuel_efficiency

x = 'prcp_pct'
# x = 'prcp_avg'
y = 'fuel_efficiency'

fig = px.scatter(data, x, y,
    color = 'airline_name',
    size = 'distance',
    size_max = 30,
    range_x = [data[x].min(), data[x].max()],
    range_y = [data[y].min(), data[y].max()],
    hover_data = ['month_name'],
    # animation_frame = 'month_name',
    )
fig.show()

<ipython-input-5-a8e5dc8476e3>:129: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  data = df.query('fuel_efficiency<1000').sort_values(['airline_name', 'month'])


In [10]:
qry = f"""
select
*
from
(
    {flights_distance}
)
where
origin_iata = "DFW"
order by monthly_prcp desc
"""
dfw = bq.qry_to_df(qry)
def report(dfw):
    print(dfw.shape)
    display(dfw.sort_values(['fuel_efficency', 'monthly_prcp'], ascending=False).head(10))
    fig = px.scatter(
    dfw.sort_values('month'),
    x = 'fuel_efficency',
    y = 'monthly_prcp',
    color = 'airline_name',
    size = 'fuel_efficency',
    size_max = 50,
    hover_name = 'origin_iata',
    log_x = True,
    log_y = True,         range_x = [100  ,6500],
    range_y = [1, 170000],
    # animation_frame = 'month',
    )
    fig.show()

report(dfw)

(94, 16)


,month,origin_iata,origin_city,origin_state,origin_station_id,airline,month_name,monthly_distance,monthly_prcp,iata_code,month_1,fuel_usage,iata_code_1,airline_1,fuel_efficency,airline_name
index,,,,,,,,,,,,,,,,
42,3,DFW,Dallas-Fort Worth,TX,USW00003927,OO,March,24357905,1969,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
36,4,DFW,Dallas-Fort Worth,TX,USW00003927,OO,April,24212316,2281,OO,April,4221,OO,Skywest Airlines Inc.,5736.156361,Skywest Airlines Inc.
54,2,DFW,Dallas-Fort Worth,TX,USW00003927,OO,February,21168576,1289,OO,February,3726,OO,Skywest Airlines Inc.,5681.314010,Skywest Airlines Inc.
51,6,DFW,Dallas-Fort Worth,TX,USW00003927,OO,June,24851213,1476,OO,June,4387,OO,Skywest Airlines Inc.,5664.739685,Skywest Airlines Inc.
23,5,DFW,Dallas-Fort Worth,TX,USW00003927,OO,May,24664339,5423,OO,May,4400,OO,Skywest Airlines Inc.,5605.531591,Skywest Airlines Inc.
57,1,DFW,Dallas-Fort Worth,TX,USW00003927,OO,January,23094965,1028,OO,January,4129,OO,Skywest Airlines Inc.,5593.355534,Skywest Airlines Inc.
78,7,DFW,Dallas-Fort Worth,TX,USW00003927,OO,July,26542222,304,OO,July,5077,OO,Skywest Airlines Inc.,5227.934213,Skywest Airlines Inc.
85,8,DFW,Dallas-Fort Worth,TX,USW00003927,OO,August,26457201,174,OO,August,5152,OO,Skywest Airlines Inc.,5135.326281,Skywest Airlines Inc.
46,11,DFW,Dallas-Fort Worth,TX,USW00003927,OO,November,24004667,1598,OO,November,4927,OO,Skywest Airlines Inc.,4872.065557,Skywest Airlines Inc.


In [9]:
AA = f"""
select
*
from
(
    {flights_distance}
)
where
origin_iata = "DFW"
and
airline_name = "American Airlines Inc."
"""
AA=bq.qry_to_df(AA)
fig = px.bar(AA, x='month', y='fuel_efficency', color='monthly_prcp', labels={'fuel_efficency', 'month'}, height=400)
fig.show()
#the least fuel efficent month was the month with the most rain

In [11]:
summary = f"""
select
*
from(
{flights_distance}
)
"""
summary = bq.qry_to_df(summary)
summary.describe()

,month,monthly_distance,monthly_prcp,fuel_usage,fuel_efficency
count,9620.000000,9.620000e+03,9620.000000,9620.000000,9620.000000
mean,6.088046,3.752661e+07,1932.683576,121110.055509,1331.213936
std,3.435358,2.494030e+07,7458.809445,118007.341716,1995.478196
min,1.000000,3.455868e+06,0.000000,3726.000000,179.501972
25%,3.000000,1.291227e+07,97.000000,16728.000000,224.987732
50%,6.000000,2.645720e+07,348.000000,59382.000000,403.099882
75%,9.000000,6.019464e+07,1090.000000,252401.000000,523.002753
max,12.000000,8.610554e+07,191484.000000,334771.000000,5920.735294


In [8]:
def report(f):
    print(f.shape)
    display(f.sort_values(['fuel_efficency', 'monthly_prcp'], ascending=False).head(10))
    fig = px.scatter(
    f.sort_values('month'),
    x = 'fuel_efficency',
    y = 'monthly_prcp',
    color = 'airline_name',
    size = 'fuel_efficency',
    size_max = 50,
    hover_name = 'origin_iata',
    log_x = True,
    log_y = True,         range_x = [100  ,6500],
    range_y = [1, 200000],
    animation_frame = 'month',
    )
    fig.show()

report(f)

(9620, 16)


,month,origin_iata,origin_city,origin_state,origin_station_id,airline,month_name,monthly_distance,monthly_prcp,iata_code,month_1,fuel_usage,iata_code_1,airline_1,fuel_efficency,airline_name
index,,,,,,,,,,,,,,,,
161,3,ORD,Chicago,IL,USW00094846,OO,March,24357905,21770,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
279,3,MSP,Minneapolis,MN,USW00014922,OO,March,24357905,12919,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
283,3,SLC,Salt Lake City,UT,USW00024127,OO,March,24357905,12731,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
298,3,DEN,Denver,CO,USW00003017,OO,March,24357905,12191,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
477,3,IAH,Houston,TX,USW00012960,OO,March,24357905,7116,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
792,3,SEA,Seattle,WA,USW00024233,OO,March,24357905,4083,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
969,3,MKE,Milwaukee,WI,USW00014839,OO,March,24357905,3255,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
1129,3,DTW,Detroit,MI,USW00094847,OO,March,24357905,2737,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.
1199,3,CVG,Covington,KY,USW00093814,OO,March,24357905,2573,OO,March,4114,OO,Skywest Airlines Inc.,5920.735294,Skywest Airlines Inc.


## Results

**Discussion:**



 To get better visualizations I limited my tables to single airports and individual airlines so I can see the correlation between fuel_efficency and weather.  Moreover, the most fuel-efficient airline was SkyWest, frontier, and spirit. The data may be a little skewed and maybe under evaluating airlines that fly with a heavier weight. In future projects, I would like to include different planes flown and weights for the planes. This simplicity of not including weight is probably the reason why smaller airlines like SkyWest and Atlantic southwest are so far ahead.



New discussion looking at the bar graph and the scatter plot it seems like airlines are more fuel efficient the less it rains and spend the most on fuel during rainy seasons



**Obstacles**



From my current findings, I want to incorporate wind data and the weight of planes to see if I can get more accurate results.



